In [2]:
import pandas as pd
import ast
from datasets import Dataset

In [ ]:
import json
chunksize = 10000
datasets_list = []

dataset_train= pd.read_csv(r"train_40k_stratified.csv",converters={
        "input_ids": json.loads,
        "attention_mask": json.loads
    })
dataset_train = Dataset.from_pandas(dataset_train)

In [4]:
dataset_train = dataset_train.rename_column("label", "labels")
dataset_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [5]:
ex = dataset_train[0]
print(type(ex["input_ids"]), ex["input_ids"].shape)  

<class 'torch.Tensor'> torch.Size([512])


In [6]:
chunksize = 1000
datasets_list = []

dataset_test= pd.read_csv(r"test_10k_stratified.csv", converters={
        "input_ids": json.loads,
        "attention_mask": json.loads
    })
dataset_test = Dataset.from_pandas(dataset_test)

In [7]:
dataset_test = dataset_test.rename_column("label", "labels")
dataset_test.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [8]:
ex = dataset_test[0]
print(type(ex["input_ids"]), ex["input_ids"].shape)  

<class 'torch.Tensor'> torch.Size([512])


In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [12]:
import torch 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [13]:
from transformers import BertForSequenceClassification, AutoModelForSequenceClassification
base_model =AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels=2
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
base_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from peft import LoraConfig, get_peft_model
lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS",
    target_modules=["query", "value"]
)

In [ ]:

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

trainable params: 1,181,186 || all params: 110,664,964 || trainable%: 1.0674


In [ ]:
import evaluate
import numpy as np
accuracy = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": acc["accuracy"], "eval_f1": f1["f1"]}

In [ ]:
from transformers import (
    TrainingArguments,
    Trainer)
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=10,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    push_to_hub=False,
    report_to="none",
    fp16=True,
    gradient_accumulation_steps=2,
    dataloader_num_workers=0,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    disable_tqdm=False
    )

In [ ]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11308\4179968431.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [20]:
import torch
torch.multiprocessing.set_start_method("spawn", force=True)

In [21]:
print("Taille vocabulaire modèle :", base_model.get_input_embeddings().weight.shape[0])

Taille vocabulaire modèle : 30522


In [ ]:

print("Starting training...")
trainer.train()

Starting training...


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.174900,0.371399,0.849929,0.852900
2,0.139400,0.413701,0.853073,0.855900
3,0.130000,0.419734,0.853261,0.856100


TrainOutput(global_step=3000, training_loss=0.19537875207265218, metrics={'train_runtime': 2753.308, 'train_samples_per_second': 43.584, 'train_steps_per_second': 1.09, 'total_flos': 3.200875905024e+16, 'train_loss': 0.19537875207265218, 'epoch': 3.0})

In [ ]:

results = trainer.evaluate()
print("Evaluation results:", results)

Evaluation results: {'eval_f1': 0.8532607705294275, 'eval_loss': 0.41973447799682617, 'eval_accuracy': 0.8561, 'eval_runtime': 82.6752, 'eval_samples_per_second': 120.955, 'eval_steps_per_second': 12.096, 'epoch': 3.0}


In [ ]:

trainer.save_model("lora_bert_model")
print("Model saved successfully.")

Model saved successfully.
